# Machine Learning 2021 - 2022 <br>
Week 3 - Workshop on Lnear Regression <br>
Lecturer: Bashir Al-Diri <br>

# Machine Learning Workshop: Week 3

This week we will use the theory about linear regression that we have learned last and this week. We will look at polynomial regression using the least squares solution derived before. Here, we will also see interesting findings about model-selection and see the over and under-fitting problem directly in our data set.

Please read over the whole notebook. It contains several excercises that you have to complete. 

# Linear Regression in Matrix Form

<img src='regression.png'>
We want to find a linear function (line) that best fits the data

## Linear Functions in vector form

More formally, we want to fit a linear function of the following form:
    $$\hat y = w_0 + \sum_i w_i x_i $$
$\beta_0$ is the offset and $w_i$ defines the slope for the ith input.
We can also write the output $\hat{y}$ in vector form
$$\hat{y} =  \boldsymbol x^T\boldsymbol w \textrm{ with } \boldsymbol w = \left[\begin{array}{c}w_0 \\ \vdots \\ w_d \end{array} \right] \textrm{ and } \boldsymbol x = \left[\begin{array}{c}1 \\ x_1 \\ \vdots \\ x_d \end{array} \right]$$
Note that we prepended a one to the $\boldsymbol x$-vector which will multiply with the offset $w_0$ when computing the scalar product.

## Matrices for multiple outputs

We will now consider multiple samples $\boldsymbol x_i$, where we will prepend again a $1$ to create the ${\boldsymbol x}_{i} = \left[\begin{array}{c}1 \\ x_{i,1} \\ \vdots \\ x_{i,d} \end{array} \right]$ 
 vector. We can stack all ${\boldsymbol x_{i}}$  in a matrix ${\boldsymbol X} = \left[\begin{array}{c}{\boldsymbol x}_{d}\\ \vdots \\ {\boldsymbol x}_{n}  \end{array} \right].$
The output $\hat y_i$ for each sample can also be  subsumed in a vector 
$\hat{\boldsymbol y} = \left[\begin{array}{c}\hat{y}_{1}\\ \vdots \\ \hat{ y}_{n}  \end{array} \right] = \left[\begin{array}{c} {\boldsymbol x_1}^T {\boldsymbol w} \\ \vdots \\ {\boldsymbol x_n}^T {\boldsymbol w}  \end{array} \right] = {\boldsymbol X} {\boldsymbol{w}}.$ Hence, the computation of all output values can be written as matrix vector product


## Lets do it in python...

We will consider a 1-dimensional problem as illustrated below. We are given 10 training samples and we want to fit
a line to these samples. Our line has 2 parameters, $\beta_0$ and $\beta_1$. Lets first look at the data and how we can compute a prediction using hand-picked 
$\beta_0$ and $\beta_1$ values. 

In [23]:
import pandas as pd
import matplotlib.pyplot

import numpy as np

data_train = pd.read_csv('regression_train.csv')
data_test = pd.read_csv('regression_test.csv')

data_train

,Unnamed: 0,x,y
0,0,0.507979,-2.885948
1,1,2.081478,-8.107921
2,2,-2.090953,-9.628462
3,3,0.108276,3.871277
4,4,3.929470,29.992393
5,5,3.962931,35.265511
6,6,-3.744147,-47.535566
7,7,-2.927571,-21.047461
8,8,-4.485328,-90.654904
9,9,-0.591902,-5.330597


### Get the training data as numpy arrays

In [24]:
x_train = data_train['x']
y_train = data_train['y']

x_test = data_test['x']
y_test = data_test['y']


### Plot the training data

In [25]:
%matplotlib notebook
## get input output vectors from the data frame and plot the data
import matplotlib.pyplot as plt


plt.clf()
plt.plot(x_train,y_train, 'bo')
plt.plot(x_test,y_test, 'g')
plt.legend(('training points', 'ground truth'))
#plt.hold(True)
plt.savefig('trainingdata.png')
plt.show()

<IPython.core.display.Javascript object>

## Preparing the data matrix
As a first step, lets construct the $\tilde{\boldsymbol{X}}$ matrix 

In [26]:
X = np.column_stack((np.ones(x_train.shape), x_train))
X

array([[ 1.        ,  0.50797903],
       [ 1.        ,  2.08147823],
       [ 1.        , -2.09095261],
       [ 1.        ,  0.10827605],
       [ 1.        ,  3.92946954],
       [ 1.        ,  3.96293089],
       [ 1.        , -3.7441469 ],
       [ 1.        , -2.92757122],
       [ 1.        , -4.48532797],
       [ 1.        , -0.59190156]])

# Matrix calculus

Now, we want to compute the derivatives of functions $f(\boldsymbol{x})$ that take several input values, i.e., a vector $\boldsymbol{x}$. As we have several input values $x_i$, we also need to derive the function with respect to each of them. The derivative of $f$ is therefore not a single value, but a vector that contains the derivative w.r.t each variable, i.e.
$$\frac{\partial f(\boldsymbol x)}{\partial \boldsymbol x} = \left[\frac{\partial f(\boldsymbol x)}{\partial  x_1}, \dots, \frac{\partial f(\boldsymbol x)}{\partial x_d}\right].$$

The rules for the differentation are similar to the scalar rules, with small differences as listed below:
* **Constants**:
$$ \frac{\partial \boldsymbol a}{\partial \boldsymbol x} = \boldsymbol 0$$ 
* **Linear term:** 
$$ \frac{\partial \boldsymbol A \boldsymbol x}{\partial \boldsymbol x} = \boldsymbol A$$
* **Quadratic terms**:
    $$ \frac{\partial \boldsymbol x^T \boldsymbol x}{\partial \boldsymbol x} = 2 \boldsymbol x^T$$
    
    $$ \frac{\partial \boldsymbol x^T \boldsymbol A \boldsymbol x}{\partial \boldsymbol x} = 2 \boldsymbol x^T \boldsymbol A,$$
    if $\boldsymbol A$ is a symmetrix matrix.
* **Linearity**: Still holds...
* **Chain Rule**: The chain rule is easy to generalize to the vector case.
For a composition of functions, i.e., $y = \boldsymbol f(\boldsymbol g( \boldsymbol x))$, we can again introduce an auxiliary variable $\boldsymbol u = \boldsymbol g( \boldsymbol x)$. The derivative of  $y = f(\boldsymbol g( \boldsymbol x))$ is then given by
$$ \frac{\partial f(\boldsymbol g(\boldsymbol x)) }{\partial \boldsymbol x} = \frac{\partial  f(\boldsymbol u)}{\partial \boldsymbol u} \frac{\partial \boldsymbol u}{\partial \boldsymbol x}.$$

**Example:** We want to compute the derivative of $$ (\boldsymbol B \boldsymbol x)^T(\boldsymbol B \boldsymbol x)$$. This function can be decomposed in $f(\boldsymbol u) = \boldsymbol u^T \boldsymbol u$ and $\boldsymbol u =\boldsymbol g( \boldsymbol x) = \boldsymbol B \boldsymbol x$, with $h(\boldsymbol x) = f(\boldsymbol g(\boldsymbol x))$.
* Compute derivative of $f$: $$ \frac{\partial  f(\boldsymbol u)}{\partial \boldsymbol u} = \frac{\partial  (\boldsymbol u^T \boldsymbol u)}{\partial \boldsymbol u}= 2 \boldsymbol u^T.$$
* Compute derivative of $\boldsymbol u$: $$ \frac{\partial  \boldsymbol u}{\partial \boldsymbol x} = \frac{\partial  (\boldsymbol B \boldsymbol x)}{\partial \boldsymbol x} = \boldsymbol B.$$
* Final Result:
$$ \frac{\partial f(\boldsymbol g( \boldsymbol x)) }{\partial \boldsymbol x} = \frac{\partial  f( \boldsymbol u)}{\partial \boldsymbol u} \frac{\partial \boldsymbol u}{\partial \boldsymbol x} = 2 \boldsymbol u^T \cdot \boldsymbol B =  2 \underbrace{\boldsymbol x^T \boldsymbol B^T}_{u^T} \boldsymbol B$$

### Excercise 1

Using the chain rule, compute the derivative of   
$$E(\boldsymbol x) = (\boldsymbol a - 5\boldsymbol x)^T \boldsymbol A(\boldsymbol{a} - 5 \boldsymbol x).$$ Set the derivative to zero and compute the minimum. 
Plot $E(\boldsymbol x)$ and $\frac{\partial E(\boldsymbol x)}{\partial x}$ as 3D plot. For $x_0$ and $x_1$, use an interval of $[-5, 5]$ for the plot using $51$ partitions for each dimension. Confirm your finding of the minimum in the plot. Note that $\boldsymbol x$ is a 2x1 vector in this equation.

$$E(\boldsymbol x) = \boldsymbol a^T \boldsymbol A \boldsymbol a - 10 \boldsymbol a^T \boldsymbol A \boldsymbol x +  25 \boldsymbol x^T \boldsymbol A  \boldsymbol x $$

$$\frac{\partial E(\boldsymbol x)}{\partial x} = 10 \boldsymbol a^T \boldsymbol A + 50 \boldsymbol x^T \boldsymbol A   $$

$$ \boldsymbol x = \frac{1}{5} \boldsymbol a$$ 



In [27]:
A = np.array([[1, 0.5], [0.5, 1]])
a = np.array([[1], [0]])

# specify data points for x0 and x1 (from - 5 to 5, using 51 uniformly distributed points)
x0Array = np.linspace(-5, 5, 51)
x1Array = np.linspace(-5, 5, 51)

Earray = np.zeros((51,51))

for i in range(0,50):
    for j in range(0,50):
        
        x = np.array([[x0Array[i]], [x1Array[j]]])
        tmp = a - 5 * x
        
        Earray[i,j] = tmp.transpose().dot(A).dot(tmp)


from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure()
ax = fig.gca(projection='3d')

x0Grid, x1Grid = np.meshgrid(x0Array, x1Array)

# Plot the surface.
surf = ax.plot_surface(x0Grid, x1Grid, Earray, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.xlabel('beta0')
plt.ylabel('beta1')
plt.savefig('errorfunction.png')
plt.show()


<IPython.core.display.Javascript object>

C:\Users\sjl10\AppData\Local\Temp/ipykernel_20288/1627975277.py:25: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')


# Minimizing the SSE function

In order to compute the minimum of the function, we need to compute its derivative and set it to zero.
The SSE function is given by: $$SSE(\boldsymbol{w}) = (\boldsymbol{y} - \boldsymbol{X}\boldsymbol{w})^T  (\boldsymbol{y} - {\boldsymbol{X}}{\boldsymbol{w}}).$$
We can again use the chain rule and introduce the auxiliary variable $\boldsymbol u = (\boldsymbol{y} - {\boldsymbol{X}}{\boldsymbol{w}})$. The SSE is then given by 
$$SSE({\boldsymbol w}) = f(\boldsymbol g({\boldsymbol w})),$$
with $f(\boldsymbol{u}) = \boldsymbol{u}^T\boldsymbol{u}$ and $\boldsymbol{u} = \boldsymbol{g}({ \boldsymbol{w}})= (\boldsymbol{y} - {\boldsymbol{X}}{\boldsymbol{w}}) $

* Compute derivative of $f$: $$ \frac{\partial  f(\boldsymbol u)}{\partial \boldsymbol u} = \frac{\partial  (\boldsymbol u^T \boldsymbol u)}{\partial \boldsymbol u}= 2 \boldsymbol u^T.$$
* Compute derivative of $\boldsymbol u$: $$ \frac{\partial  \boldsymbol u}{\partial \boldsymbol{w}} = \frac{\partial (\boldsymbol{y} - \boldsymbol{X}\boldsymbol{w}) }{\partial  \boldsymbol{w}} = - \boldsymbol{X}.$$
* Final Result:
$$ \frac{\partial f(\boldsymbol g( \boldsymbol{w})) }{\partial \boldsymbol{w} } = \frac{\partial  f( \boldsymbol u)}{\partial \boldsymbol u} \frac{\partial \boldsymbol u}{\partial \boldsymbol{w}} = - 2 \boldsymbol u \boldsymbol{X} = - 2(\boldsymbol{y} - \boldsymbol{X}\boldsymbol{w})^T\boldsymbol{X}. $$

### Gradient Descent to find the parameters
We have the gradient vector and so can find the optimal parameters using Gradient Descent algorithm.
You can now write the gradient descent algorithm code to find the weights.

In [28]:
# put you code here
import numpy as np
w_old = [np.random.randn(),np.random.randn()]
w_new = [np.random.randn(),np.random.randn()]
epsilon = 0.00001
alpha = 0.01
def grad(w):
    #temp = -2*(y_train-X.dot(w))
    #return temp.transpose().dot(X)
    #(𝐴𝐵)𝑇=𝐵𝑇𝐴𝑇
    return -2*X.transpose().dot(y_train-X.dot(w))

states = []
while abs(np.linalg.norm(np.array(w_new)-np.array(w_old)))>epsilon:
    w_old = w_new
    w_new = w_old - alpha*grad(w_old)
    states.append(w_new)
    print('w_new={}'.format(w_new))
print("Local minimum occurs at {}".format(w_new))


w_new=[-2.22698389 19.15779711]
w_new=[-2.8576533   5.32676588]
w_new=[-4.26114127 14.4634326 ]
w_new=[-4.79009097  8.30952669]
w_new=[-5.61322588 12.35860574]
w_new=[-6.00856258  9.61831701]
w_new=[-6.50293791 11.41095571]
w_new=[-6.78192503 10.1893083 ]
w_new=[-7.0845161  10.98180634]
w_new=[-7.27508029 10.43627287]
w_new=[-7.46298877 10.78587884]
w_new=[-7.59059279 10.54168269]
w_new=[-7.70854762 10.69542673]
w_new=[-7.79291884 10.58574251]
w_new=[-7.86754478 10.65304035]
w_new=[-7.92287148 10.60353416]
w_new=[-7.97035052 10.63278828]
w_new=[-8.00643237 10.61029063]
w_new=[-8.03676008 10.62287392]
w_new=[-8.06020441 10.61255305]
w_new=[-8.07963067 10.61787775]
w_new=[-8.0948256 10.6130819]
w_new=[-8.10729325 10.61527661]
w_new=[-8.11712473 10.61300996]
w_new=[-8.12513723 10.61387501]
w_new=[-8.13149101 10.61278023]
w_new=[-8.13664519 10.61309371]
w_new=[-8.14074816 10.6125507 ]
w_new=[-8.14406582 10.61264434]
w_new=[-8.14671387 10.61236657]
w_new=[-8.14885036 10.61237878]
w_new=[-8.

In [29]:
# We can now compute the error on the training set
error = y_train -X.dot(w_new)
SSE = error.dot(error)
SSE

2679.2717977603947

In the following you will see that the weights can be explicitly found by setting the gradient to zero.

### Solving for the parameters

We now need to set the derivative to $0$, i.e., 
$$\frac{\partial SSE( {\boldsymbol{w}})}{\partial {\boldsymbol{w}}} = \boldsymbol 0.$$
Hence, the derivative w.r.t every dimension should be zero. Now, we solve for ${\boldsymbol{w}}$ using the following steps:
* Cancel constant factors:

$$(\boldsymbol{y} - {\boldsymbol{X}}{\boldsymbol{w}})^T {\boldsymbol{X}} = \boldsymbol 0. $$
* Transpose on both sides (using $(\boldsymbol{A} \boldsymbol{B})^T = \boldsymbol{B}^T \boldsymbol{A}^T$): 

$${\boldsymbol{X}}^T(\boldsymbol{y} - {\boldsymbol{X}}{\boldsymbol{w}}) = \boldsymbol 0 $$
* Multiply out brackets

$${\boldsymbol{X}}^T\boldsymbol{y} - {\boldsymbol{X}}^T {\boldsymbol{X}}{\boldsymbol{w}} = \boldsymbol 0 $$
* Bring ${\boldsymbol{X}}^T\boldsymbol{y}$ on other side

$${\boldsymbol{X}}^T {\boldsymbol{X}}{\boldsymbol{w}} = {\boldsymbol{X}}^T\boldsymbol{y}$$

* Multiply both sides by the inverse of ${\boldsymbol{X}}^T {\boldsymbol{X}}$ (multiply on the left)

$${\boldsymbol{w}} = ({\boldsymbol{X}}^T {\boldsymbol{X}})^{-1} {\boldsymbol{X}}^T\boldsymbol{y}$$


## The least squares solution

The least squares solution is given by:
$${\boldsymbol{w}} = ({\boldsymbol{X}}^T {\boldsymbol{X}})^{-1} {\boldsymbol{X}}^T\boldsymbol{y}$$

* ** Note: ** The term $({\boldsymbol{X}}^T{\boldsymbol{X}})^{-1}{\boldsymbol{X}}^T$ is called the pseudo inverse of matrix ${\boldsymbol{X}}$. It is used to invert non-square matrices that could not be inverted otherwise.

## Exercise 2 - Implementation in python

We now want to implement the least squares solution that is given above in python. First, construct the Xtilde matrix.
Subsequently implement the least squares solution. Try to avoud the linalg.inv method but use the numerically more stable linalg.solve 
method instead. 
* Plot again the training data, ground truth and prediction with the optimal least squares solution
* What is the SSE of the optimal solution? 
* Compare your solution to the solution we have found in previous worshop with the grid search. How much do we 
do better now?

In [30]:
# Check wether we still have Xtilde (otherwise we need to rerun all scripts)
X
#X = np.column_stack((np.ones(x_train.shape), x_train))


array([[ 1.        ,  0.50797903],
       [ 1.        ,  2.08147823],
       [ 1.        , -2.09095261],
       [ 1.        ,  0.10827605],
       [ 1.        ,  3.92946954],
       [ 1.        ,  3.96293089],
       [ 1.        , -3.7441469 ],
       [ 1.        , -2.92757122],
       [ 1.        , -4.48532797],
       [ 1.        , -0.59190156]])

In [31]:
import numpy.linalg as linalg

XX = X.transpose().dot(X)

#w = np.linalg.solve(XX, X.transpose().dot(y_train))
w = np.linalg.inv(XX).dot(X.transpose().dot(y_train))

w

array([-8.15753129, 10.61195127])

In [32]:
# Lets plot our function

import matplotlib.pyplot as plt

Xtest = np.column_stack((np.ones(x_test.shape), x_test))
ytest_predicted = Xtest.dot(w)

plt.figure()
plt.plot(x_test,y_test, 'g')
plt.plot(x_test, ytest_predicted, 'r')
plt.plot(x_train,y_train, 'bo')
plt.legend(('training points', 'ground truth', 'prediction'), loc = 'lower right')
#plt.hold(True)
plt.savefig('regression_LSS.png')
plt.show()

<IPython.core.display.Javascript object>

We can now compute  the error on the training set

In [33]:
error = y_train -X.dot(w)
SSE = error.dot(error)
SSE


2679.2717977476764

In comparison to the hand-picked line parameters from above, our error is approximately half. However, we can still see that the approximation is far from perfect. The ground truth can not be represented by a line, so we will always have a rather large approximation error. 

# Polynomial Regression

Instead of fitting a line, we can also fit a polynomial. We want to fit $d$th order polynomials which are given by
$$\hat y = w_0 + \sum_{i = 1}^d w_i x^i$$
Note that, while $\hat y$ is now non-linear in $x$, it is still linear in the parameters $w_i$. Hence, we can still apply linear regression here!

 ## Setting up the data matrix
 We can still describe $\hat{y}$ as a scalar product, i.e.,
 $$\hat{y} = w_0 + \sum_{i = 1}^n w_i x^i = \boldsymbol{x}^T\boldsymbol{w}, \textrm{ with } {\boldsymbol{x}} = \left[\begin{array}{c} 1 \\ x^1 \\ x^2 \\ \vdots \\ x^d  \end{array}\right] \textrm{ and } {\boldsymbol{w}} = \left[\begin{array}{c} w_0 \\ \vdots \\ w_{d+1}  \end{array}\right]$$
 

### Setting up the data matrix in python

In python, we write a small function that does the feature expansion up to a certain degree for a given data set x.

In [34]:
def getPolynomialDataMatrix(x, degree):
    X = np.ones(x.shape)
    for i in range(1,degree + 1):
        X = np.column_stack((X, x ** i))
    return X
    
print(getPolynomialDataMatrix(x_train, 4))

[[ 1.00000000e+00  5.07979026e-01  2.58042691e-01  1.31080275e-01
   6.65860302e-02]
 [ 1.00000000e+00  2.08147823e+00  4.33255161e+00  9.01811183e+00
   1.87710034e+01]
 [ 1.00000000e+00 -2.09095261e+00  4.37208282e+00 -9.14181799e+00
   1.91151082e+01]
 [ 1.00000000e+00  1.08276052e-01  1.17237034e-02  1.26939632e-03
   1.37445222e-04]
 [ 1.00000000e+00  3.92946954e+00  1.54407309e+01  6.06738818e+01
   2.38416171e+02]
 [ 1.00000000e+00  3.96293089e+00  1.57048212e+01  6.22371212e+01
   2.46641410e+02]
 [ 1.00000000e+00 -3.74414690e+00  1.40186360e+01 -5.24878324e+01
   1.96522155e+02]
 [ 1.00000000e+00 -2.92757122e+00  8.57067324e+00 -2.50912563e+01
   7.34564398e+01]
 [ 1.00000000e+00 -4.48532797e+00  2.01181670e+01 -9.02365770e+01
   4.04740642e+02]
 [ 1.00000000e+00 -5.91901563e-01  3.50347461e-01 -2.07371210e-01
   1.22743343e-01]]


## Exercise 3: Fit Polynomials with different degrees
We now want to test different polynomials and see which one fits our data best. First, implement a function 
that computes the optimal beta values given the input data x, output data y and the desired degree of the polynomial. Reuse the getPolynomialDataMatrix given above in your function.

In [35]:
import numpy.linalg as linalg

def getWeightsForPolynomialFit(x,y,degree):
    X = getPolynomialDataMatrix(x, degree)

    XX = X.transpose().dot(X)
    w = np.linalg.solve(XX, X.transpose().dot(y))
    #w = np.linalg.inv(XX).dot(X.transpose().dot(y))

    return w

Given your getBetaForPolynomialFit function, plot the fitted function for a polynomial 1st, 2nd, 3rd and 4th degree.
Can we now fit the structure of the function better?

In [36]:
# Lets plot our polynomials function

import matplotlib.pyplot as plt
import numpy.linalg as linalg

plt.figure()
plt.plot(x_test,y_test, 'g')
plt.plot(x_train,y_train, 'bo')

w1 = getWeightsForPolynomialFit(x_train,y_train,1)
Xtest1 = getPolynomialDataMatrix(x_test, 1)
ytest1 = Xtest1.dot(w1)
plt.plot(x_test, ytest1, 'r')

w2 = getWeightsForPolynomialFit(x_train,y_train,2)
Xtest2 = getPolynomialDataMatrix(x_test, 2)
ytest2 = Xtest2.dot(w2)
plt.plot(x_test, ytest2, 'g')

w3 = getWeightsForPolynomialFit(x_train,y_train,3)
Xtest3 = getPolynomialDataMatrix(x_test, 3)
ytest3 = Xtest3.dot(w3)
plt.plot(x_test, ytest3, 'm')

w4 = getWeightsForPolynomialFit(x_train,y_train,4)
Xtest4 = getPolynomialDataMatrix(x_test, 4)
ytest4 = Xtest4.dot(w4)
plt.plot(x_test, ytest4, 'c')

plt.legend(('training points', 'ground truth', '$x$', '$x^2$', '$x^3$', '$x^4$'), loc = 'lower right')

plt.savefig('polynomial.png')

## errors on test dataset Bashir
error1 = y_test-ytest1
SSE1 = error1.dot(error1)

error2 = y_test-ytest2
SSE2 = error2.dot(error2)

error3 = y_test-ytest3
SSE3 = error3.dot(error3)

error4 = y_test-ytest4
SSE4 = error4.dot(error4)

SSE1, SSE2, SSE3, SSE4



<IPython.core.display.Javascript object>

(43755.70731418699, 38010.71049272058, 1405.8967619762366, 1754.7879609542251)

As we can see, the predictions for 3rd and 4th order are already quite good. Lets see what happens if we increase 
the order of the polynomial. Repeat the same plots for for example 7, 10 and 12th order polynomials.

In [37]:
# Lets plot our polynomials function

import matplotlib.pyplot as plt
import numpy.linalg as linalg

plt.figure()
plt.plot(x_test,y_test, 'g')
plt.plot(x_train,y_train, 'bo')

w7 = getWeightsForPolynomialFit(x_train,y_train,7)
Xtest7 = getPolynomialDataMatrix(x_test, 7)
ytest7 = Xtest7.dot(w7)
plt.plot(x_test, ytest7, 'r')

w10 = getWeightsForPolynomialFit(x_train,y_train,10)
Xtest10 = getPolynomialDataMatrix(x_test, 10)
ytest10 = Xtest10.dot(w10)
plt.plot(x_test, ytest10, 'c')

w12 = getWeightsForPolynomialFit(x_train,y_train,12)
Xtest12 = getPolynomialDataMatrix(x_test, 12)
ytest12 = Xtest12.dot(w12)
plt.plot(x_test, ytest12, 'm')

plt.ylim((-200, 200))
plt.legend(('training points', 'ground truth', '$x^{7}$', '$x^{10}$', '$x^{12}$'), loc = 'lower right')

plt.savefig('polynomial1.png')

## errors on test dataset Bashir
error7 = y_test-ytest7
SSE7 = error7.dot(error7)

error10 = y_test-ytest10
SSE10 = error10.dot(error10)

error12 = y_test-ytest12
SSE12 = error12.dot(error12)

SSE7, SSE10, SSE12

<IPython.core.display.Javascript object>

(121441.64194897415, 64343707.88472931, 926153978.6540107)

We can clearly see that the prediction performance of our polynomials degrade. Why is that? 
This effect is called overfitting. We have too many data dimensions and too little data in order to fit the polynomial. Overfitting can have 2 bad effects:
* We fit the noise in the data
* The function 'does what it wants' between the data points. It is underspecified what the function should do between the data points. 

However, for the training data, the fit is actually almost perfect. Thats another characteristic of overfitting!

## Exercise 4: Evaluating the Models
We now want to evaluate the learned models to see which one works best. To do so, we compare polynomials of order
1 to 12. Compute the squarred error on the training and on the test set for each of these polynomials and plot them 
as a function of the degree of the polynomial. Due to the huge differences in the error, use a log-scale plot for the y-axis (see pyplot.semilogy) What are your observations? 

In [38]:
SSEtrain = np.zeros((11,1))
SSEtest = np.zeros((11,1))
MSSEtrain = np.zeros((11,1))
MSSEtest = np.zeros((11,1))

# Feel free to use the functions getWeightsForPolynomialFit and getPolynomialDataMatrix
for i in range(1,12):
    
    Xtrain = getPolynomialDataMatrix(x_train, i) 
    Xtest = getPolynomialDataMatrix(x_test, i)
    
    w = getWeightsForPolynomialFit(x_train, y_train, i)  
    
    MSSEtrain[i - 1] = np.mean((Xtrain.dot(w) - y_train)**2)
    MSSEtest[i - 1] = np.mean((Xtest.dot(w) - y_test)**2)
    
    errortrain = y_train - Xtrain.dot(w) 
    errortest = y_test - Xtest.dot(w)
    SSEtrain[i-1] = errortrain.dot(errortrain)
    SSEtest[i-1] = errortest.dot(errortest)
    

In [40]:
# Execute this...
import matplotlib.pyplot as plt

plt.figure();
plt.semilogy(range(1,12), SSEtrain)
plt.semilogy(range(1,12), SSEtest)
plt.legend(('SSE on training set', 'SSE on test set'))
plt.savefig('polynomial_evaluation.png')
plt.show()

<IPython.core.display.Javascript object>

## Overfitting / Under-Fitting
We can see that, while the training error decreases with a larger degree of the polynomial, the test set error
significantly increases (which is the one we are interested in). This is a typical behavior we get for overfitting. 
We can make the following conclusions:
* In order to fit the data well, we have to find the right model complexity
* **Over-fitting:** The model-complexity is too high (degree > 4). We fit the noise and not the data
* **Under-fitting:** The model-complexity is too low (degree < 3). We can not represent the data well enough.
* For choosing the optimal model, we always have to consider the error on an independent test set not on the training set
* On the training set, the error can be arbitrarily good. This is only an indication that the algorithm has learned the example by heart, not that it can generalize to new samples.